In [2]:
import pandas as pd
import re
import glob

# Define the file pattern to match
file_pattern = '*_full.csv'

# Create a list of all the matching files
sellers_files = glob.glob(file_pattern)

# Initialize an empty list to store DataFrames
dfs = []

# Loop through the sellers_files list and read each CSV file into a DataFrame
for seller_file in sellers_files:
    df = pd.read_csv(seller_file)
    dfs.append(df)

# Create df_sellers_aggregate
df_sellers_aggregate = pd.concat(dfs, ignore_index=True)

# Stock nb of duplicates
number_of_duplicates = df_sellers_aggregate.duplicated()

# Remove the duplicates
df_sellers_aggregate.drop_duplicates(inplace=True)

#Transform "media_condition" and "sleeve_condition"
df_sellers_aggregate["media_condition"] = df_sellers_aggregate["media_condition"].str.replace(r'\([^)]*\)', '', regex=True)
df_sellers_aggregate["sleeve_condition"] = df_sellers_aggregate["sleeve_condition"].str.replace(r'\([^)]*\)', '', regex=True)

#Transform "nb_for_sale"
df_sellers_aggregate["nb_for_sale"] = df_sellers_aggregate["nb_for_sale"].str.extract(r'(\d+)')

#Transform "nb_for_sale" and handle "--" values
price_columns = ["current_lowest_price","low_price", "median_price", "high_price"]

for column in price_columns:
    df_sellers_aggregate[f"{column}"] = df_sellers_aggregate[f"{column}"].str[1:]
    df_sellers_aggregate[f"{column}"] = df_sellers_aggregate[f"{column}"].replace('-', float('nan'))

#Transform "avg_rating" and handle "--" values
df_sellers_aggregate['avg_rating'] = df_sellers_aggregate['avg_rating'].str.split(' / ').str.get(0)
df_sellers_aggregate['avg_rating'] = df_sellers_aggregate['avg_rating'].replace('--', float('nan'))

#Change type
float_data = ["price", "current_lowest_price","low_price", "median_price", "high_price","avg_rating"]
for column in float_data:
    df_sellers_aggregate[f"{column}"] = df_sellers_aggregate[f"{column}"].astype(float)

int_data = ["have", "want", "ratings","nb_for_sale"]
for column in int_data:
    df_sellers_aggregate[f"{column}"] = df_sellers_aggregate[f"{column}"].astype(int)

df_sellers_aggregate['last_sold_date'] = pd.to_datetime(df_sellers_aggregate['last_sold_date'], format='%b %d, %Y', errors='coerce')

# Function to extract the release ID  from the release_url
def extract_number(release_url):
    match = re.search(r'/(\d+)-', release_url)
    if match:
        return match.group(1)
    else:
        return None

# Apply the function to create the 'release_id' column
df_sellers_aggregate['release_id'] = df_sellers_aggregate['release_url'].apply(extract_number)

# Reorder the columns to place 'release_id' as the first column
df_sellers_aggregate = df_sellers_aggregate[['release_id'] + [col for col in df_sellers_aggregate.columns if col != 'release_id']]

In [3]:
df_sellers_aggregate

,release_id,release_url,media_condition,sleeve_condition,price,img_url,format_detail,nb_for_sale,current_lowest_price,have,want,avg_rating,ratings,last_sold_date,low_price,median_price,high_price
0,6104789,https://www.discogs.com/release/6104789-Guy-Ma...,Near Mint,Near Mint,10.00,https://i.discogs.com/7n-fM3Z955-ItygPW_oS9arZ...,"CD, Album",14,3.39,27,2,NaN,0,2022-05-30,5.00,5.00,5.00
1,6195652,https://www.discogs.com/release/6195652-Mozart...,Mint,Mint,10.00,https://i.discogs.com/qvkYVgZI-w2yhk07c4_Kyawu...,"CD, Album, Reissue, Remastered, Digipack",11,4.95,51,18,3.00,1,2023-01-19,8.78,9.38,9.99
2,15454460,https://www.discogs.com/release/15454460-Johan...,Mint,Mint,10.00,https://i.discogs.com/T2Z8GKhO84efR87DYsp4Txzy...,CD,7,3.00,79,2,5.00,1,2023-08-19,2.21,2.75,14.99
3,14248726,https://www.discogs.com/release/14248726-Orlan...,Mint,Mint,10.00,https://i.discogs.com/-fECcp7-aBn4A3sBBkKd8SZa...,"CD, Stereo",4,2.28,16,5,3.00,2,2022-08-10,19.94,19.94,19.94
4,11093459,https://www.discogs.com/release/11093459-Arman...,Mint,Good Plus,14.00,https://i.discogs.com/0S7zG0bHRZHU1Yr83Ndv0DzG...,"CD, Album",6,20.00,17,4,3.00,2,2023-09-12,18.00,29.98,51.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,16142459,https://www.discogs.com/release/16142459-zovie...,Near Mint,Near Mint,13.00,https://i.discogs.com/Esv09crpb4mFO9-TWOkJS3Wo...,"Vinyl, LP, 45 RPM, Album, Limited Edition, Rem...",5,25.00,135,100,4.68,22,2023-11-08,19.49,23.00,42.00
737,22782650,https://www.discogs.com/release/22782650-Supre...,Mint,Mint,9.55,https://i.discogs.com/hwyDumQGOIQmSFg8KlaBT-Zs...,"CD, Album, Limited Edition",6,14.00,31,6,4.71,7,2024-01-05,9.50,14.55,15.46
738,13510605,https://www.discogs.com/release/13510605-Visio...,Mint,Near Mint,9.55,https://i.discogs.com/Ax5wkCCnQdrnh2MRlyUXlgL4...,"CDr, Album, Limited Edition, DVD Boxset",1,13.55,0,4,4.00,1,2020-10-04,10.00,10.00,10.00
739,13280668,https://www.discogs.com/release/13280668-Diffe...,Near Mint,Near Mint,10.50,https://i.discogs.com/kT3-u_yoj3rLs_0sPIMr_j0g...,"CD, Album, Limited Edition",9,7.00,26,4,2.33,3,2021-05-17,2.00,5.00,6.95
